# <font color="#418FDE" size="6.5" uppercase>**open And Globals**</font>

>Last update: 20251221.
    
By the end of this Lecture, you will be able to:
- Use open with context managers to read from and write to files safely, specifying modes and encodings. 
- Inspect and, when necessary, modify global and local namespaces using globals and locals in controlled scenarios. 
- Use __import__ for dynamic imports while understanding its risks and preferring higher-level alternatives when possible. 


## **1. Safe File Opening**

### **1.1. Text and Binary Modes**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_09/Lecture_B/image_01_01.jpg?v=1766327531" width="250">



>* Text mode interprets file bytes as characters
>* Runtime handles encodings and cross-platform line endings

>* Binary mode reads and writes raw bytes exactly
>* Prevents text decoding changes that corrupt binary data

>* Choosing text or binary affects safety, correctness
>* Be explicit about mode to prevent data corruption



In [ ]:
#@title Python Code - Text and Binary Modes

# Demonstrate text mode reading and writing with simple string content.
# Demonstrate binary mode reading and writing with raw byte content.
# Compare how text and binary modes display the same underlying bytes.

text_filename = "example_text.txt"

binary_filename = "example_binary.bin"

text_content = "Hello, world!\nSecond line here."

binary_content = bytes([72, 101, 108, 108, 111, 44, 32, 255])

with open(text_filename, "w", encoding="utf-8") as text_file:

    text_file.write(text_content)

with open(binary_filename, "wb") as binary_file:

    binary_file.write(binary_content)

print("Reading file using text mode shows characters and lines.")

with open(text_filename, "r", encoding="utf-8") as text_file:

    text_data = text_file.read()

print("Text mode content:", repr(text_data))

print("Reading file using binary mode shows raw byte values.")

with open(binary_filename, "rb") as binary_file:

    binary_data = binary_file.read()

print("Binary mode content:", binary_data)

print("Length of binary data bytes:", len(binary_data))

with open(text_filename, "rb") as mixed_file:

    mixed_data = mixed_file.read()

print("Same text file bytes when read binary:", mixed_data)



### **1.2. With Statement Basics**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_09/Lecture_B/image_01_02.jpg?v=1766327576" width="250">



>* with automatically opens and safely closes files
>* prevents resource leaks, especially in long programs

>* with blocks clearly mark when files are usable
>* they aid readability, maintenance, and team trust

>* with blocks close files even on errors
>* prevents corruption, leaks, and shows careful resource use



In [ ]:
#@title Python Code - With Statement Basics

# Demonstrate safe file opening using with statement basics.
# Compare manual close and automatic close using with blocks.
# Show that with blocks close files even after raised exceptions.

# First, open a file manually and remember to close it yourself.
manual_file = open("example_manual.txt", mode="w", encoding="utf-8")
manual_file.write("Manual open requires manual close.\n")
manual_file.close()

# Now, open a file using a with block for automatic closing.
with open("example_with.txt", mode="w", encoding="utf-8") as safe_file:
    safe_file.write("With statement closes this file automatically.\n")

# Show that the file is closed after leaving the with block.
print("File closed after with block:", safe_file.closed)

# Demonstrate that with still closes the file after an exception.
try:
    with open("example_error.txt", mode="w", encoding="utf-8") as error_file:
        error_file.write("Writing before an intentional error.\n")
        raise ValueError("Simulated processing problem inside with block.")
except ValueError as problem:
    print("Caught error, message was:", problem)

# Check that the file from the error block is closed properly.
print("File closed after error with block:", error_file.closed)



### **1.3. Encoding and newline basics**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_09/Lecture_B/image_01_03.jpg?v=1766327624" width="250">



>* Encoding maps characters to stored file bytes
>* Always choose and specify encoding, usually UTF-8

>* Mismatched encodings can silently corrupt shared text
>* Always choose and document a standard encoding like UTF-8

>* Newline bytes differ across systems; normalization helps portability
>* Sometimes exact newline bytes matter; control translation behavior



In [ ]:
#@title Python Code - Encoding and newline basics

# Demonstrate explicit encoding when writing and reading simple text files.
# Show how newline translation changes line endings across different operating systems.
# Help beginners choose encoding and newline options for portable text files.

from pathlib import Path
from textwrap import dedent

base_folder = Path("encoding_newline_demo_folder")
base_folder.mkdir(exist_ok=True)

utf8_path = base_folder / "greeting_utf8.txt"
latin1_path = base_folder / "greeting_latin1.txt"

text_content = "Café on Main Street, mile 3.5."

with utf8_path.open(mode="w", encoding="utf-8", newline="\n") as file_utf8:
    file_utf8.write(text_content + "\nSecond line here.\n")

with latin1_path.open(mode="w", encoding="latin-1", newline="\r\n") as file_latin1:
    file_latin1.write(text_content + "\r\nSecond line here.\r\n")

print("UTF-8 file written with explicit encoding and Unix style newlines.\n")
print("Latin-1 file written with explicit encoding and Windows style newlines.\n")

with utf8_path.open(mode="r", encoding="utf-8", newline=None) as file_utf8_read:
    normalized_lines = list(file_utf8_read)

with latin1_path.open(mode="r", encoding="latin-1", newline=None) as file_latin1_read:
    normalized_lines_latin = list(file_latin1_read)

print("Reading with newline=None normalizes endings to single \n character.\n")
print("UTF-8 normalized lines count and sample:")
print(len(normalized_lines), repr(normalized_lines[0]))

print("Latin-1 normalized lines count and sample:")
print(len(normalized_lines_latin), repr(normalized_lines_latin[0]))

with latin1_path.open(mode="r", encoding="latin-1", newline="") as file_latin1_raw:
    raw_content = file_latin1_raw.read()

print("Raw Latin-1 content shows original Windows style newline bytes:")
print(repr(raw_content))




## **2. Globals And Locals**

### **2.1. Exploring Module Globals**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_09/Lecture_B/image_02_01.jpg?v=1766327675" width="250">



>* Module globals form a shared top-level workspace
>* They map names to objects like a toolbox

>* Use globals to adapt code dynamically
>* Discover features, tests, and metadata from names

>* Interpret globals carefully; many names are incidental
>* Follow project conventions to avoid fragile dependencies



In [ ]:
#@title Python Code - Exploring Module Globals

# Demonstrate exploring module global namespace using globals dictionary.
# Show how top level names appear inside the globals mapping.
# Filter and print selected global names and their associated object types.

PI_APPROX_VALUE = 3.14  # Define simple constant representing approximate circle ratio value.

favorite_city_name = "Boston"  # Define simple string representing favorite United States city.

miles_to_work_distance = 12  # Define integer representing daily commute distance in miles.


def describe_globals_briefly():
    """Return filtered global names and types for demonstration purposes only."""
    global_items = globals().items()  # Get view of global names and objects mapping.
    filtered = []  # Prepare list for storing selected name and type description pairs.
    for name, value in global_items:
        if name.startswith("__"):
            continue
        if name in {"describe_globals_briefly", "print"}:
            continue
        filtered.append((name, type(value).__name__))
    return filtered


selected_globals = describe_globals_briefly()  # Collect filtered global names and types list.

print("Module globals overview (name -> type):")  # Print heading describing following global information.

for name, type_name in selected_globals:
    print(f"{name} -> {type_name}")  # Print each selected global name and corresponding object type.




### **2.2. Locals Inside Functions**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_09/Lecture_B/image_02_02.jpg?v=1766327715" width="250">



>* Function locals form a temporary workspace dictionary
>* locals() shows current names and their values

>* Use locals to inspect variables while debugging
>* Safely evaluate expressions using current local scope

>* locals inside functions act as read-only snapshots
>* use explicit data structures instead of mutating locals



In [ ]:
#@title Python Code - Locals Inside Functions

# Demonstrate locals inside simple functions workspace clearly.
# Show how parameters and temporary variables appear in locals dictionary.
# Highlight that modifying locals dictionary inside functions is unreliable.

def show_survey_step(response_text, valid_count):
    cleaned_response = response_text.strip().lower()
    should_keep = bool(cleaned_response)
    print("Locals snapshot inside function call now:")
    print(locals())


def try_modify_locals():
    feet_value = 10
    inches_value = 6
    print("Before locals modification attempt, values are:")
    print("feet_value:", feet_value, "inches_value:", inches_value)


    local_view = locals()
    local_view["feet_value"] = 20
    local_view["inches_value"] = 0
    print("After locals modification attempt, values are:")
    print("feet_value:", feet_value, "inches_value:", inches_value)


print("Calling show_survey_step with example response now:")
show_survey_step("  Yes  ", 3)


print("\nCalling try_modify_locals to demonstrate modification risk:")
try_modify_locals()



### **2.3. Risks of modifying namespaces**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_09/Lecture_B/image_02_03.jpg?v=1766327765" width="250">



>* Directly changing namespaces is powerful but fragile
>* Small tweaks can cause widespread, hard-to-find bugs

>* Dynamic name injection reduces clarity and readability
>* Hidden namespace changes hinder debugging and team trust

>* Untrusted inputs changing namespaces can undermine safety
>* Hidden namespace tweaks hurt refactoring and long-term maintenance



In [ ]:
#@title Python Code - Risks of modifying namespaces

# Demonstrate how changing globals can create confusing and fragile behavior.
# Show a safe version that uses a normal global assignment pattern.
# Show a risky version that mutates globals dynamically using the globals dictionary.

SAFE_RATE_MPH_TO_MPS = 0.44704  # Safe constant defined clearly at module level.

RISKY_RATE_NAME = "MPH_TO_MPS"  # Name that will be injected into globals dynamically.


def safe_speed_conversion(mph_value):  # Safe function using explicit global constant.
    meters_per_second = mph_value * SAFE_RATE_MPH_TO_MPS  # Clear dependency on constant.
    return meters_per_second  # Return converted speed using safe constant.


def risky_inject_rate(new_rate):  # Risky function that mutates the global namespace.
    globals()[RISKY_RATE_NAME] = new_rate  # Dynamically injects or overwrites global name.
    print("Injected global name:", RISKY_RATE_NAME, "with value:", new_rate)


def risky_speed_conversion(mph_value):  # Risky function depending on injected global name.
    meters_per_second = mph_value * globals()[RISKY_RATE_NAME]  # Hidden fragile dependency.
    return meters_per_second  # Return converted speed using injected rate.


print("Safe conversion uses:", SAFE_RATE_MPH_TO_MPS, "meters per second per mile per hour.")

print("Safe result for 60 mph:", safe_speed_conversion(60))

risky_inject_rate(0.40)  # Accidentally inject wrong rate, maybe from configuration or user input.

print("Risky result for 60 mph:", risky_speed_conversion(60))

print("Globals now contain MPH_TO_MPS:", "MPH_TO_MPS" in globals())

print("Actual MPH_TO_MPS value:", globals()["MPH_TO_MPS"])



## **3. Safe Dynamic Imports**

### **3.1. Core Import Mechanics**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_09/Lecture_B/image_03_01.jpg?v=1766327808" width="250">



>* Python tracks loaded modules in a registry
>* Importing loads, initializes, and executes module code

>* Special function imports modules by string name
>* Used to load optional plugins only when needed

>* Imports run module code and trigger side effects
>* Cached modules reuse mutable state, affecting safety



In [ ]:
#@title Python Code - Core Import Mechanics

# Demonstrate basic import mechanics using repeated imports and module attributes.
# Show that Python caches imported modules inside sys.modules registry.
# Illustrate that importing executes top level code only during first import.

import importlib
import sys


print("Before importing, math in registry:", "math" in sys.modules)
math_module_first = importlib.import_module("math")


print("After first import, math in registry:", "math" in sys.modules)
print("First import, math id value:", id(math_module_first))


math_module_second = importlib.import_module("math")
print("Second import, math id value:", id(math_module_second))


print("Same object reused for imports:", math_module_first is math_module_second)



### **3.2. String Based Imports**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_09/Lecture_B/image_03_02.jpg?v=1766327912" width="250">



>* Module names come from runtime strings, not code
>* Enables flexible plugins but adds complexity and risk

>* Framework loads team modules from config strings
>* Flexible plugin style, but fragile at runtime

>* Treat module name strings as powerful, risky input
>* Whitelist, validate, and verify modules to stay safe



In [ ]:
#@title Python Code - String Based Imports

# Demonstrate dynamic string based imports using simple tool modules.
# Show safe mapping from user choices to internal module names.
# Verify imported modules provide expected attributes before using them.

import types as builtin_types_module
import importlib as builtin_importlib_module

# Create a fake module representing a length conversion tool.
length_tools_module = builtin_types_module.ModuleType("length_tools_module")
setattr(length_tools_module, "tool_name", "Length conversion tool")
setattr(length_tools_module, "describe", lambda: "Converts inches to feet safely.")


# Create another fake module representing a temperature conversion tool.
temp_tools_module = builtin_types_module.ModuleType("temp_tools_module")
setattr(temp_tools_module, "tool_name", "Temperature conversion tool")
setattr(temp_tools_module, "describe", lambda: "Converts Fahrenheit to Celsius.")


# Register these fake modules inside sys modules dictionary.
import sys as builtin_sys_module
builtin_sys_module.modules["length_tools_module"] = length_tools_module
builtin_sys_module.modules["temp_tools_module"] = temp_tools_module


# Map safe user choices to internal module names.
SAFE_TOOL_MAP = {"length": "length_tools_module", "temperature": "temp_tools_module"}


# Simulate a user choice coming from configuration or input.
user_choice_string = "length"
print("Requested tool choice string:", user_choice_string)


# Look up the real module name using the safe mapping.
module_name_string = SAFE_TOOL_MAP.get(user_choice_string)
print("Resolved internal module name:", module_name_string)


# Dynamically import the module using its name string.
imported_module_object = builtin_importlib_module.import_module(module_name_string)
print("Imported module tool name:", imported_module_object.tool_name)


# Verify the module provides the expected describe attribute.
if hasattr(imported_module_object, "describe"):
    print("Tool description text:", imported_module_object.describe())
else:
    print("Selected module missing required describe attribute.")




### **3.3. Safer Dynamic Imports**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_09/Lecture_B/image_03_03.jpg?v=1766327956" width="250">



>* Treat every dynamic import as untrusted code
>* Centralize imports and restrict modules to approved lists

>* Never feed raw user input into imports
>* Map user choices to a vetted module registry

>* Handle dynamic import errors with logging and fallbacks
>* Combine controls, validation, and error handling for safety



In [ ]:
#@title Python Code - Safer Dynamic Imports

# Demonstrate safer dynamic imports using a simple plugin registry.
# Show how to avoid importing arbitrary user provided module names.
# Handle missing plugins gracefully while keeping the program running.

import importlib
from types import ModuleType

# Define a safe registry mapping user choices to internal module names.
PLUGIN_REGISTRY = {
    "sales_tax": "math",  # Pretend math handles sales tax calculations.
    "shipping_cost": "statistics",  # Pretend statistics handles shipping costs.
}

# Simulate untrusted user input that selects a plugin by friendly name.
user_choice = "sales_tax"  # Try changing this to "shipping_cost" or "unknown".

# Look up the internal module name using the safe registry mapping.
internal_module_name = PLUGIN_REGISTRY.get(user_choice)

# Only attempt dynamic import when the choice exists in the registry.
if internal_module_name is None:
    print("Unknown plugin requested, using safe default behavior.")
    plugin_module: ModuleType | None = None
else:
    try:
        plugin_module = importlib.import_module(internal_module_name)
        print(f"Loaded plugin module safely: {internal_module_name}.")
    except ImportError as error:
        print(f"Failed importing plugin, using fallback instead: {error}.")
        plugin_module = None

# Use the plugin module only when it loaded successfully, otherwise use fallback.
if plugin_module is not None:
    example_value = 100.0  # Example dollars amount for a purchase.
    tax_rate = 0.07  # Seven percent sales tax rate.
    tax_amount = example_value * tax_rate
    print(f"Calculated example tax amount: {tax_amount} dollars.")
else:
    print("Fallback behavior used, no plugin calculations performed.")



# <font color="#418FDE" size="6.5" uppercase>**open And Globals**</font>


In this lecture, you learned to:
- Use open with context managers to read from and write to files safely, specifying modes and encodings. 
- Inspect and, when necessary, modify global and local namespaces using globals and locals in controlled scenarios. 
- Use __import__ for dynamic imports while understanding its risks and preferring higher-level alternatives when possible. 

In the next Module (Module 10), we will go over 'Advanced And Misc'